In [ ]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()

w.dbutils.library.restartPython()

In [ ]:
from databricks.connect import DatabricksSession
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from datetime import datetime, date
import pandas as pd
import polars as pl
import sys

spark = DatabricksSession.builder.getOrCreate()

root_dir = Path.cwd().parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

ppa_details_path = root_dir / "utils" / "ppa_details.csv"
ppa_details_df = pd.read_csv(ppa_details_path)
ppa_details = spark.createDataFrame(ppa_details_df)

In [ ]:
from utils.extract import extract_deal_info
from utils.transform import clean_and_aggregate_earnings

start_date = "2025-01-01"
end_date = "2030-06-30"

sim_complete_deal_info = extract_deal_info()

In [ ]:
aurora_df = spark.table("exploration.default.earnings_aurora_scenarios")
aurora_earnings = clean_and_aggregate_earnings(
    df=aurora_df,
    filter_status=True,
    status_list=["Approved", "Confirmed", "Pending" ],
    deal_info_df=sim_complete_deal_info,
    curve_name="aurora",
    end_date=end_date,
    is_aurora=True,
    agg_by_month=True
)

combined_earnings = aurora_earnings

In [ ]:
scenario_prices = (
    spark.table("exploration.denise_ng.latest_forward_price_trace")
    .filter(F.col("Scenario").isin(["Central", "Messy"]))
    .select(
        F.col("Scenario").alias("scenario"),
        F.concat(F.col("region"), F.lit(1)).alias("regionid"),
        F.col("DateTime").alias("interval_date_time"),
        F.col("Interval_date").alias("interval_date"),
        F.col("Nominal_$").alias("rrp")
    )
).cache()

In [ ]:
combined_earnings_df = pl.from_pandas(combined_earnings.toPandas())

In [ ]:
(
    combined_earnings_df.filter(
        pl.col("total_earnings").is_not_null() & (pl.col("year") >= 2025)
    )
    .filter(
        pl.col("curve") == "Central"
    )
    .with_columns(
        pl.when(pl.col("buy_sell") == "Buy")
        .then(pl.col("total_income"))
        .otherwise(-pl.col("total_cost"))
        .alias("floating_earnings"),
        pl.when(pl.col("buy_sell") == "Buy")
        .then(-pl.col("total_cost"))
        .otherwise(pl.col("total_income"))
        .alias("fixed_earnings"),
        pl.when(pl.col("deal_id") == 888_888)
        .then(pl.lit("Hypothetical"))
        .otherwise(pl.col("status"))
        .alias("status"),
    )
    .sort(
        ["year", "month", "group", "regionid", "deal_id"]
    )
    .select(
        pl.col("curve").alias("Curve Name"),
        pl.col("year").alias("Year"),
        pl.col("month").alias("Month"),
        pl.col("deal_id").alias("Deal ID"),
        pl.col("deal_name").alias("Deal Name"),
        # Implement product_name logic based on interval_date
        pl.when(pl.col("deal_id") == 888_888)
        .then(
            pl.when(pl.col("interval_date") < date(2026, 1, 1))
            .then("Pre-Refi $27/MWh")
            .when((pl.col("interval_date") >= date(2026, 1, 1)) & (pl.col("interval_date") < date(2031, 1, 1)))
            .then("Post-Refi $23.5/MWh")
            .otherwise("Post-Refi $22.5/MWh + 2.5%")
        )
        .otherwise(pl.col("product_name"))
        .alias("Product Name"),
        pl.col("instrument").alias("Instrument"),
        pl.when(pl.col("status").is_in(["Approved", "Confirmed"]))
        .then(pl.lit("Contracted"))
        .otherwise(pl.lit("Hypothetical"))
        .alias("Status"),
        pl.col("buy_sell").alias("Buy/Sell"),
        pl.col("strategy").alias("Strategy"),
        pl.col("regionid").alias("Region ID"),
        pl.col("group").alias("Group"),
        pl.col("volume_mwh").alias("Volume (MWh)"),
        pl.col("fixed_earnings").alias("Fixed Earnings"),
        (pl.col("fixed_earnings") / pl.col("volume_mwh")).alias("VW Fixed Rate"),
        pl.col("floating_earnings").alias("Floating Earnings"),
        (pl.col("floating_earnings") / pl.col("volume_mwh")).alias("VW Floating Rate"),
        pl.col("twp").alias("TW Floating Price"),
        pl.col("total_earnings").alias("Total Earnings"),
    )
    .write_csv("aurora_central.csv")
)